In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
np.random.seed(7)

df = pd.read_csv("./boston_housing.csv")
dataset = df.values
np.random.shuffle(dataset)

In [3]:
X = dataset[:, 0:13]
Y = dataset[:, 13]
#標準化
X -= X.mean(axis=0)
X /= X.std(axis=0)

X_train, Y_train = X[:404], Y[:404] 
X_test, Y_test = X[404:], Y[404:]  

In [4]:
def build_deep_model():
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation="relu"))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam", 
                  metrics=["mae"])
    return model

In [5]:
#kfold
k = 4
nb_val_samples = len(X_train) // k
nb_epochs = 80
mse_scores = []
mae_scores = []

In [6]:
for i in range(k):
    X_val = X_train[i*nb_val_samples: (i+1)*nb_val_samples]
    Y_val = Y_train[i*nb_val_samples: (i+1)*nb_val_samples]

    X_train_p = np.concatenate(
            [X_train[:i*nb_val_samples],
            X_train[(i+1)*nb_val_samples:]], axis=0)
    Y_train_p = np.concatenate(
            [Y_train[:i*nb_val_samples],
            Y_train[(i+1)*nb_val_samples:]], axis=0)
    model = build_deep_model()

    model.fit(X_train_p, Y_train_p, epochs=nb_epochs, 
              batch_size=16, verbose=0)

    mse, mae = model.evaluate(X_val, Y_val, verbose=0)
    mse_scores.append(mse)
    mae_scores.append(mae)

In [7]:
print("MSE_val: ", np.mean(mse_scores))
print("MAE_val: ", np.mean(mae_scores))

mse, mae = model.evaluate(X_test, Y_test, verbose=0)    
print("MSE_test: ", mse)
print("MAE_test: ", mae)

MSE_val:  18.889856338500977
MAE_val:  2.842442919712256
MSE_test:  8.588650048947802
MAE_test:  2.2924087561813056


#### save model note

In [ ]:
'''
# 儲存模型結構
json_str = model.to_json()
with open("Modelname.config", "w") as text_file:
    text_file.write(json_str)
# 儲存模型權重
model.save_weights("Modelname.weight")
'''
'''
# 儲存模型結構和權重way2
model.save("modelname.h5")
'''

In [ ]:
'''
# 載入模型結構
from tensorflow.keras.models import model_from_json

model = Sequential()
with open("Modelname.config", "r") as text_file:
    json_str = text_file.read()
model = model_from_json(json_str)
# 載入權重
model.load_weights("Modelname.weight", by_name=False)
# 編譯模型
model.compile(loss="binary_crossentropy", optimizer="adam", 
              metrics=["accuracy"])
#use
'''
'''
# 載入模型way2
from tensorflow.keras.models import load_model

model = Sequential()
model = load_model("Modelname.h5")
# 編譯模型
model.compile(loss="binary_crossentropy", optimizer="adam", 
              metrics=["accuracy"])
'''